In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car
from Functions.py_functions.gps_importer import *
from Functions.py_functions.tire_model.tire_model_utils import *

In [ ]:
v_avg = 15
max_beta = 25
max_delta = 15
size = 9
use_lin = True

In laymans terms the goal of this script is to determine the 'schwarzschild' β angle, or more accuratly β and δ combinations assuming a dβ of zero. Essentially the event horizion of fucking up and spinning out, to determine what α angles are actually even relivant to the conversation, bill cobb says your crazy to need over 10 deg (I'm unclear of if he is talking about α or δ) and on the otherhand Bobby do be looking a little sideways (https://www.youtube.com/watch?v=DkUvy43231M)
And I choose to assume a dβ of zero because there isnt any other practical way of solving this that I could think of

In [ ]:
if use_lin:
    # delta and beta with linear spacing
    beta = np.linspace(-max_beta, max_beta, (2 * size + 1))
    delta = np.linspace(0, max_delta, (size + 1))
    delta_two = np.concatenate((-delta[::-1], delta[1:]))
else:
    # delta and beta with log spacing
    beta = np.geomspace(0.1, max_beta, size)
    beta = np.concatenate((-beta[::-1], np.array([0]), beta))
    delta = np.geomspace(0.1, max_delta, size)
    delta_two = np.concatenate((-delta[::-1], np.array([0]), delta))
    delta = np.concatenate((np.array([0]), delta))

In [ ]:
# preppare for the sweep
delta_v, beta_v = np.meshgrid(delta, beta)
inds = np.argwhere(np.full(delta_v.shape, True))
ay = np.zeros(delta_v.shape)
yaw = np.zeros(delta_v.shape)
bruh = np.zeros(delta_v.shape)
stability = np.ones(delta_v.shape)
car = Car()
car.set_tire(H_R20_18X6_6)
car.set_lltd()

In [ ]:
# sweep
for index in inds:
    ay_it, cn_it, yaw_it, ax_it, bruh_it, _ = car.solve_for_long(v_avg, 0.0, delta_x=np.deg2rad(delta_v[index[0], index[1]]), beta_x=np.deg2rad(beta_v[index[0], index[1]]))
    ay[index[0], index[1]] = ay_it
    yaw[index[0], index[1]] = yaw_it
    bruh[index[0], index[1]] = bruh_it

In [ ]:
def stability_test(delta, beta, vel, long_g=0.0, time_step = 0.01, delta_dot_max = 30.0):
    ay_it, cn_it, yaw_it, ax_it, bruh_it, _ = car.solve_for_long(v_avg, long_g, delta_x=np.deg2rad(delta), beta_x=np.deg2rad(beta))
    pos = -1
    if beta > 0:
        pos = 1
    d_sign = np.sign(delta)
    steps = []
    omega_it, omega_v_it = ay_it / vel, ay_it / vel
    delta_it, beta_it = delta, beta
    delta_dot, ay_dot = 0, 0
    ay_int = ay_it * time_step
    steps.append([delta_it, beta_it, ay_it, yaw_it, ax_it, omega_it, omega_v_it, delta_dot, ay_dot, 0, 0, ay_int])
    i = 0
    beta_deriv = 0
    beta_dot = 0
    kp, ki, kd = -2.0, -8.0, -0.15
    k_ay_int = 0.5
    while abs(beta_it) > 1 or abs(beta_dot) > 10:
        omega_it += np.rad2deg(yaw_it) * time_step
        omega_v_it = np.rad2deg(ay_it * G / vel) * -1
        prev_beta_dot = beta_dot
        beta_dot = omega_it - omega_v_it
        beta_deriv = (beta_dot - prev_beta_dot)
        # beta_int *= 0.9 # decay
        # beta_int += beta_it * time_step
        ay_int *= 0.9 # decay
        ay_int += ay_it * time_step
        delta_dot = min(delta_dot_max, max(-delta_dot_max, delta_dot_max * (kp * (beta_dot / max_beta) + ki * (beta_it / max_beta) + kd * beta_deriv + k_ay_int * ay_int)))
        delta_it = min(max_delta, max(-max_delta, delta_dot * time_step + delta_it))
        # if delta_it == max_delta or delta_it == -max_delta:
            # delta_dot = 0
        beta_it += beta_dot * time_step
        # if delta_dot == 0 and np.sign(beta_prev - beta_it) == np.sign(beta_it):
            # return 1, steps
        ay_it, cn_it, yaw_it, ax_it, bruh_it, _ = car.solve_for_long(v_avg, long_g, delta_x=np.deg2rad(delta_it), beta_x=np.deg2rad(beta_it))
        steps.append([delta_it, beta_it, ay_it, yaw_it, ax_it, omega_it, omega_v_it, delta_dot, ay_dot, (omega_it - omega_v_it), beta_deriv, ay_int])
        if delta_it > max_delta or delta_it < -max_delta or beta_it > max_beta * 2 or beta_it < -max_beta * 2 or i > 200:
            return 0, steps
        i += 1
    return 1, steps
    
all_steps = []
all_inds = []
bad_steps = []
bad_inds = []
first = True
# for index in [[7, 3]]:
for index in inds:
    stable, steps = stability_test(delta_v[index[0], index[1]], beta_v[index[0], index[1]], v_avg)
    stability[index[0], index[1]] = stable
    # print(index, stable, np.array(steps).shape)
    
    # all_steps.append(np.array(steps))
    # bad_steps.append(np.array(steps))
    # if first:
    #     first = False
    # else:
    #     break
    if stable == 1:
        all_steps.append(np.array(steps))
        all_inds.append(index)
    else:
        bad_steps.append(np.array(steps))
        bad_inds.append(index)
    


In [ ]:
# plot the first 1 step out of steps
fig = go.Figure()
step = all_steps[82]
# print(bad_inds[160])
# step = bad_steps[0]
print(len(all_steps))
fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 0], mode='lines', name='delta'))
fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 1], mode='lines', name='beta'))
fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 2], mode='lines', name='ay'))
# fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 3], mode='lines', name='yaw'))
# fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 4], mode='lines', name='ax'))
# fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 5], mode='lines', name='omega'))
# fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 6], mode='lines', name='omega_v'))
fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 7], mode='lines', name='delta_dot'))
# fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 8], mode='lines', name='ay_dot'))
fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 9], mode='lines', name='beta_dot'))
fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 10], mode='lines', name='beta_deriv'))
fig.add_trace(go.Scatter(x=np.arange(step.shape[0]), y=step[:, 11], mode='lines', name='ay_int'))

In [ ]:
# fill out the symmetric side
ay = np.concatenate((-np.flip(ay[:, 1:], (0, 1)), ay), axis=1)
yaw = np.concatenate((-np.flip(yaw[:, 1:], (0, 1)), yaw), axis=1)
bruh = np.concatenate((np.flip(bruh[:, 1:], (0, 1)), bruh), axis=1)
stability = np.concatenate((np.flip(stability[:, 1:], (0, 1)), stability), axis=1)

In [ ]:
# plot Stability
fig = px.imshow(stability, labels=dict(x="Delta", y="Beta", color="Stability"), origin='lower', x=delta_two, y=beta, aspect="auto")
for ind, step in enumerate(all_steps):
    fig.add_trace(go.Scatter(x=step[:, 0], y=step[:, 1], name=f"{all_inds[ind]} {ind}", legendgroup=f"a{ind}", mode='markers+lines', marker={'color': step[:, 5]}, line=dict(color="green", width=2)))
    fig.add_trace(go.Scatter(x=-step[:, 0], y=-step[:, 1], name=f"{all_inds[ind]} {ind}", legendgroup=f"a{ind}", mode='markers+lines', marker={'color': -step[:, 5]}, showlegend=False, line=dict(color="green", width=2)))
for ind, step in enumerate(bad_steps):
    fig.add_trace(go.Scatter(x=step[:, 0], y=step[:, 1], mode="lines", name=f"{bad_inds[ind]} {ind}", legendgroup=f"b{ind}", line=dict(color="red", width=2)))
    fig.add_trace(go.Scatter(x=-step[:, 0], y=-step[:, 1], mode="lines", name=f"{bad_inds[ind]} {ind}", legendgroup=f"b{ind}", showlegend=False, line=dict(color="red", width=2)))
    pass
fig.update_layout(template="plotly_dark", title_text="2D MMD")
fig.show()

In [ ]:
# plot Ay
fig = px.imshow(ay, labels=dict(x="Delta", y="Beta", color="Ay (m/s^2)"), origin='lower', x=delta_two, y=beta, aspect="auto")
fig.update_layout(template="plotly_dark", title_text="2D MMD")
fig.show()

In [ ]:
# plot Yaw accel
fig = px.imshow(yaw, labels=dict(x="Delta", y="Beta", color="Yaw Accel (rad/s^2)"), origin='lower', x=delta_two, y=beta, aspect="auto")
fig.update_layout(template="plotly_dark", title_text="2D MMD")
fig.show()

In [ ]:
# plot solve iterations
fig = px.imshow(bruh, labels=dict(x="Delta", y="Beta", color="Solve Iterations"), origin='lower', x=delta_two, y=beta, aspect="auto")
fig.update_layout(template="plotly_dark", title_text="2D MMD")
fig.show()

In [ ]:
# plot Yaw accel
delta_sp, beta_sp = np.meshgrid(delta_two, beta)
sa = beta_sp - delta_sp
print(sa.shape)
print(yaw.shape)
fig = px.imshow(sa, labels=dict(x="Delta", y="Beta", color="Approx Front SA (deg)"), origin='lower', x=delta_two, y=beta, aspect="auto")
fig.update_layout(template="plotly_dark", title_text="2D MMD")
fig.show()

In [ ]:
# create a traditional MMD plot
def delim_mmd(mask, ay_it1, yaw_it1):
    inds_raw = np.argwhere(mask)[:, 0]
    inds = np.concatenate((np.array([-2]), inds_raw, np.array([mask.shape[0] + 1])))
    delim_ind = np.where((inds[:-1] + 1 != inds[1:]))[0] + 1
    if delim_ind.shape[0] < 2:
        return ay_it1 * G, yaw_it1
    ay_it2, yaw_it2, ax_it2 = [], [], []
    for i in range(len(delim_ind)-1):
        ay_it2.extend(ay_it1[inds[delim_ind[i]:delim_ind[i+1]]] * G)
        ay_it2.append(None)
        yaw_it2.extend(yaw_it1[inds[delim_ind[i]:delim_ind[i+1]]])
        yaw_it2.append(None)
    return np.array(ay_it2), np.array(yaw_it2)

fig = go.Figure()
fig.update_xaxes(title_text='Lat Acc (m/s^2)')
fig.update_yaxes(title_text='Yaw Acc (rad/sec^2)')
for i, betax in enumerate(beta):
    valid = ay[i, :] != 0
    fig.add_trace(
        go.Scatter(
            x=ay[i, valid] * G,
            y=yaw[i, valid],
            mode='lines',
            hovertext="β={:.2f}°".format(betax),
            marker=dict(color='red'), legendgroup=f"group1", showlegend=False
        ))
    ay1, yaw1 = delim_mmd((ay[i, :] * stability[i, :]) != 0, ay[i, :], yaw[i, :])
    fig.add_trace(
        go.Scatter(
            x=ay1,
            y=yaw1,
            mode='lines',
            hovertext="β={:.2f}°".format(betax),
            marker=dict(color='red'), legendgroup=f"group2", showlegend=False
        ))
for j, deltax in enumerate(delta_two):
    valid = ay[:, j] != 0
    fig.add_trace(
        go.Scatter(
            x=ay[valid, j] * G,
            y=yaw[valid, j],
            mode='lines',
            hovertext="{:.1f}° Delta".format(deltax),
            marker=dict(color='blue'), name="MMD", legendgroup=f"group1", showlegend=(j == len(delta_two)-1)
        ))
    ay1, yaw1 = delim_mmd((ay[:, j] * stability[:, j]) != 0, ay[:, j], yaw[:, j])
    fig.add_trace(
        go.Scatter(
            x=ay1,
            y=yaw1,
            mode='lines',
            hovertext="{:.1f}° Delta".format(deltax),
            marker=dict(color='blue'), name="MMD Clean", legendgroup=f"group2", showlegend=(j == len(delta_two)-1)
        ))

step = bad_steps[31]
fig.add_trace(go.Scatter(x=step[:, 2]*G, y=step[:, 3], name=f"{all_inds[161]} {161}", legendgroup=f"a{161}", mode='markers+lines', marker={'color': step[:, 5]}, line=dict(color="green", width=2)))
fig.update_layout(template="plotly_dark", title_text="2D MMD")
fig.show()